<a href="https://colab.research.google.com/github/Aryan-Ark-02/Decision_Tree/blob/main/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Decision Tree

- CART Algorithm (used in scikit learn)
- ID3

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

##Dataset

In [ ]:
data = {
    'Time_studied': [12, 7, 2, 1, 5, 1],
    'Cgpa': [5, 6, 9, 10, 2, 5],
    'If_Placed': [0, 1, 0, 0, 1, 1]
}
df = pd.DataFrame(data)
df

,Time_studied,Cgpa,If_Placed
0,12,5,0
1,7,6,1
2,2,9,0
3,1,10,0
4,5,2,1
5,1,5,1


#CART (Classification and Regression Trees) algorithm


##For Calculating mid points (for numrical columns only)

In [ ]:
def midPoints(df,column):
  mid_points = []
  column = df[column].to_list()
  column.sort()
  for value in range(len(column)-1):
    mid = (column[value] + column[value+1])/2
    mid_points.append(mid)
  return mid_points
# mid = midPoints('Cgpa')

##For splitting dataset on a particular threshold (midpoint)

In [ ]:
def split_dataset(df, column, threshold):
  df1 = df[df[column] <= threshold]
  df2 = df[df[column] > threshold]
  return df1, df2

##To calculate impurity in dataset

In [ ]:
def gini_impurity(df):
  zeros = 0
  ones = 0
  if df.empty:
    return 0
  for i in df.If_Placed:
    if i == 0:
      zeros += 1
    else:
      ones += 1
  total = zeros + ones
  gini = 1 - ((ones/total)**2 + (zeros/total)**2)
  return gini

##For calculating best cut :

- spliting dataset on a point with minimum impurity among all mid - points




In [ ]:
def best_cut(df,column):
  mid = midPoints(df,column)
  min = 1
  threshold = None # Initialize threshold
  for i in mid:
    a = split_dataset(df,column,i)
    # Removed the inner loop and directly access the two dataframes
    df1, df2 = a
    # Corrected the denominator to be the total number of rows in the current dataframe
    total_rows = df1.shape[0] + df2.shape[0]
    gini = (df1.shape[0]/total_rows)*gini_impurity(df1) + (df2.shape[0]/total_rows)*gini_impurity(df2)
    if gini < min:
      min = gini
      threshold = i
  return min,threshold

In [ ]:
best_cut(df,'Cgpa')

(0.25, 7.5)

##Split data on column and threshold with minimum impurity

- compare all columns
- choose column with lowest gini impurity
- returns column name, threshold and its gini impurity

In [ ]:
def best_split(df):
  # Initialize with default values
  selected_column = None
  cut = None
  min_gini = 1 # Initialize with the worst possible gini impurity

  for column in df.columns[:-1]:
    gini, threshold = best_cut(df, column)
    if gini < min_gini: # Compare with min_gini instead of min
      min_gini = gini
      selected_column = column
      cut = threshold
  return selected_column, cut, min_gini

best_split(df)

('Cgpa', 7.5, 0.25)

###Calculating and spliting till root node reached

In [ ]:
split = split_dataset(df,best_split(df)[0],best_split(df)[1])

### Need Further spliting


In [ ]:
split[0]

,Time_studied,Cgpa,If_Placed
0,12,5,0
1,7,6,1
4,5,2,1
5,1,5,1


leaf node

In [ ]:
split[1]

,Time_studied,Cgpa,If_Placed
2,2,9,0
3,1,10,0


In [ ]:
best_split(split[0])

('Time_studied', 9.5, 0.0)

In [ ]:
best_split(split[1])

('Time_studied', 1.5, 0.0)

In [ ]:
split = split_dataset(split[0],best_split(split[0])[0],best_split(split[0])[1])

leaf node

In [ ]:
split[0]

,Time_studied,Cgpa,If_Placed
1,7,6,1
4,5,2,1
5,1,5,1


leaf node

In [ ]:
split[1]

,Time_studied,Cgpa,If_Placed
0,12,5,0


#ID3 (Iterative Dichotomiser 3) algorithm

In [ ]:
def entropy(df):
  if df.empty:
    return 0
  counts = df['If_Placed'].value_counts()
  total = len(df)
  ent = 0
  for count in counts:
    p = count / total
    ent -= p * np.log2(p)
  return ent

In [ ]:
def weighted_entropy(df1, df2):
  total = len(df1) + len(df2)
  w_entropy = (len(df1)/total)*entropy(df1) + (len(df2)/total)*entropy(df2)
  return w_entropy


In [ ]:
def best_cut_id3(df, column):
  mids = midPoints(df, column)
  best_gain = -1
  best_thresh = None
  original_entropy = entropy(df)

  for thresh in mids:
    df1, df2 = split_dataset(df, column, thresh)
    w_ent = weighted_entropy(df1, df2)
    info_gain = original_entropy - w_ent

    if info_gain > best_gain:
      best_gain = info_gain
      best_thresh = thresh

  return best_gain, best_thresh


In [ ]:
def best_split_id3(df):
  best_feature = None
  best_threshold = None
  best_gain = -1

  for column in df.columns[:-1]:  # skip target
    gain, thresh = best_cut_id3(df, column)
    if gain > best_gain:
      best_gain = gain
      best_feature = column
      best_threshold = thresh

  return best_feature, best_threshold, best_gain


In [ ]:
print("Root split:", best_split_id3(df))

split1 = split_dataset(df, best_split_id3(df)[0], best_split_id3(df)[1])
print("Left split:", best_split_id3(split1[0]))
print("Right split:", best_split_id3(split1[1]))


Root split: ('Cgpa', 7.5, np.float64(0.4591479170272448))
Left split: ('Time_studied', 9.5, np.float64(0.8112781244591328))
Right split: ('Time_studied', 1.5, np.float64(0.0))
